In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import random
import itertools
import gc
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
import lightgbm as lgb
import catboost as cb
import category_encoders as ce
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-feb-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-feb-2021/train.csv
/kaggle/input/tabular-playground-series-feb-2021/test.csv


In [2]:
train = pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-feb-2021/test.csv')
print(f'train: {train.shape}, test: {test.shape}')
sample = pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv')
data = pd.concat([train, test], axis=0).reset_index(drop=True)

train: (300000, 26), test: (200000, 25)


In [3]:
data.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,A,B,A,A,B,D,A,E,C,...,0.881122,0.421650,0.741413,0.895799,0.802461,0.724417,0.701915,0.877618,0.719903,6.994023
1,2,B,A,A,A,B,B,A,E,A,...,0.440011,0.346230,0.278495,0.593413,0.546056,0.613252,0.741289,0.326679,0.808464,8.071256
2,3,A,A,A,C,B,D,A,B,C,...,0.914155,0.369602,0.832564,0.865620,0.825251,0.264104,0.695561,0.869133,0.828352,5.760456
3,4,A,A,A,C,B,D,A,E,G,...,0.934138,0.578930,0.407313,0.868099,0.794402,0.494269,0.698125,0.809799,0.614766,7.806457
4,6,A,B,A,A,B,B,A,E,C,...,0.382600,0.705940,0.325193,0.440967,0.462146,0.724447,0.683073,0.343457,0.297743,6.868974


In [4]:
cat_col = []
num_col = []

for col in data.columns:
    if data[col].dtype=='object':
        cat_col.append(col)
    else:
        if col != 'id':
            num_col.append(col)
            
oh_col = []
for col in cat_col:
    data[col], _ = data[col].factorize()
    
    if data[col].nunique() > 2:
        oh_col.append(col)
        
oh_enc = ce.OneHotEncoder(cols=oh_col)
ohe_df = oh_enc.fit_transform(data[oh_col])
data = pd.concat([data, ohe_df], axis=1).reset_index(drop=True)

/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [5]:
for ccol in tqdm(['cat1', 'cat9', 'cat2', 'cat0', 'cat3']):
    for ncol in num_col:
        new_col = 'groupby_' + ccol + '_' + ncol + '_max'
        data[new_col] = data.groupby([ccol])[ncol].transform('max')
        
        new_col = 'groupby_' + ccol + '_' + ncol + '_min'
        data[new_col] = data.groupby([ccol])[ncol].transform('min')
        
        new_col = 'groupby_' + ccol + '_' + ncol + '_median'
        data[new_col] = data.groupby([ccol])[ncol].transform('median')

for (ccol1, ccol2) in itertools.combinations(['cat1', 'cat9', 'cat2', 'cat0', 'cat3'], 2):
    for ncol in ['cont9', 'cont8', 'cont5', 'cont0', 'cont13', 'cont11', 'cont6']:
        new_col = 'groupby_' + ccol1 + '_' + ccol2 + '_' + ncol + '_max'
        data[new_col] = data.groupby([ccol1, ccol2])[ncol].transform('max')

        new_col = 'groupby_' + ccol1 + '_' + ccol2 + '_' + ncol + '_min'
        data[new_col] = data.groupby([ccol1, ccol2])[ncol].transform('min')

        new_col = 'groupby_' + ccol1 + '_' + ccol2 + '_' + ncol + '_median'
        data[new_col] = data.groupby([ccol1, ccol2])[ncol].transform('median')

  0%|          | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
def frequency_enc(df, enc_col):
    for c in tqdm(enc_col):
        temp = df[c].value_counts().to_dict()
        new_col = c + '_' + 'freq_enc'
        df[new_col] = df[c].map(temp) / len(df[c])
        
    return df

data = frequency_enc(data, cat_col)

In [ ]:
############# training settings ################
SEED = 2021
N_FOLDS = 6
NUM_BOOST_ROUNDS = 10000
EARLY_STOPPING_ROUNDS = 100

random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)

def rmse(y_true, y_pred):
    return round(np.sqrt(mean_squared_error(y_true, y_pred)), 5)

params = {'objective': 'regression', 
          'metric': 'rmse', 
          'num_leaves': 40, 
          'learning_rate': 0.005, 
          'bagging_fraction': 0.5926185908890315, 
          'feature_fraction': 0.4, 
          'verbosity': -1, 'seed': 2021, 
          'feature_pre_filter': False, 
          'lambda_l1': 7.649261020958673, 
          'lambda_l2': 7.825454920039041, 
          'bagging_freq': 5, 
          'min_child_samples': 50}

col_ = ['cont8',
        'cont0',
        'cont13',
        'cont11',
        'cont9',
        'cont5',
        'cont6',
        'cont1',
        'cont3',
        'cont10',
        'groupby_cat1_cat2_cont8_max',
        'groupby_cat1_cat9_cont13_median',
        'cont7',
        'cont12',
        'cont2',
        'groupby_cat1_cat0_cont9_max',
        'cont4',
        'groupby_cat9_cat3_cont11_min',
        'cat9',
        'cat8',
        'cat8_3',
        'cat5',
        'groupby_cat1_cat0_cont9_min',
        'groupby_cat9_cat3_cont8_min',
        'groupby_cat1_cat2_cont9_max',
        'groupby_cat1_cat2_cont5_max',
        'groupby_cat2_cat0_cont8_min',
        'cat6',
        'groupby_cat1_cat2_cont5_min',
        'groupby_cat9_cat0_cont8_min',
        'groupby_cat9_cat0_cont11_max',
        'groupby_cat9_cat0_cont11_min',
        'groupby_cat1_cat0_cont9_median',
        'cat6_1',
        'groupby_cat9_cat2_cont8_max',
        'cat2',
        'cat5_4']

In [ ]:
# from https://www.kaggle.com/c/tabular-playground-series-feb-2021/discussion/216576
def create_stratified_folds_for_regression(data_df, n_splits=5, random_state=0):
    """
    @param data_df: training data to split in Stratified K Folds for a continous target value
    @param n_splits: number of splits
    @return: the training data with a column with kfold id
    """
    data_df['kfold'] = -1
    # randomize the data
    data_df = data_df.sample(frac=1).reset_index(drop=True)
    # calculate the optimal number of bins based on log2(data_df.shape[0])
    num_bins = np.int(np.floor(1 + np.log2(len(data_df))))
    print(f"Num bins: {num_bins}")
    # bins value will be the equivalent of class value of target feature used by StratifiedKFold to 
    # distribute evenly the classed over each fold
    data_df.loc[:, "bins"] = pd.cut(pd.to_numeric(data_df['target'], downcast="signed"), bins=num_bins, labels=False)
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    # set the fold id as a new column in the train data
    for f, (t_, v_) in enumerate(kf.split(X=data_df, y=data_df.bins.values)):
        data_df.loc[v_, 'kfold'] = f

    # drop the bins column (no longer needed)
    data_df = data_df.drop("bins", axis=1)

    return data_df

def kfold_splits(n_splits, train_df):
    """
    Returns a collection of (fold, train indexes, validation indexes)
    @param n_splits: number of splits
    @param train_df: training data
    @return: a collection of (fold, train indexes, validation indexes)
    """
    all_folds = list(range(0, n_splits))
    kf_splits = []
    for fold in range(0, n_splits):
        train_folds = [x for x in all_folds if x != fold]
        trn_idx = train_df[train_df.kfold!=fold].index
        val_idx = train_df[train_df.kfold==fold].index
        kf_splits.append((fold, trn_idx, val_idx))
    return kf_splits

In [ ]:
train = data[data['target'].isnull()==False]
test = data[data['target'].isnull()==True].drop('target', axis=1)
print(f'train: {train.shape}, test: {test.shape}')


X = train.drop(['id', 'target'], axis=1)
y = train['target']
test_id = test['id'].values
train_df = create_stratified_folds_for_regression(train, n_splits=N_FOLDS, random_state=SEED)
cv_list = kfold_splits(n_splits=N_FOLDS, train_df=train_df)

del train_df
gc.collect()

In [ ]:
X = X[col_]
test = test[col_]

In [ ]:
oof_train = np.zeros(len(train))
oof_score = []

# for i, (train_idx, val_idx) in enumerate(kf.split(X, y)):
for i, train_idx, val_idx in cv_list:
    print('[Fold %d/%d]' % (i + 1, N_FOLDS))
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.values[train_idx], y.values[val_idx]

    lgb_train = lgb.Dataset(X_train, label=y_train)
    lgb_val = lgb.Dataset(X_val, label=y_val)

    model = lgb.train(params, 
                      lgb_train, 
                      num_boost_round=NUM_BOOST_ROUNDS,
                      valid_names=["train", "valid"],
                      valid_sets=[lgb_train, lgb_val],
                      early_stopping_rounds=EARLY_STOPPING_ROUNDS,
#                       categorical_feature=cat_col,
                      verbose_eval=1000)

#     if i == 0:
#         importance_df = pd.DataFrame(model.feature_importance(importance_type='gain'), index=X.columns, columns=['importance'])
#     else:
#         importance_df += pd.DataFrame(model.feature_importance(importance_type='gain'), index=X.columns, columns=['importance'])

    oof_train[val_idx] = model.predict(X_val, num_iteration=model.best_iteration)
    oof_score.append(rmse(y_val, oof_train[val_idx]))
    
    if i == 0:
        pred = model.predict(test, num_iteration=model.best_iteration)
    else:
        pred += model.predict(test, num_iteration=model.best_iteration)
        
pred /= N_FOLDS

In [ ]:
print(f'OOF RMSE score: {rmse(oof_train, y.values)}')
print(f'OOF Mean RMSE score: {np.mean(oof_score)}')

In [ ]:
sub = pd.DataFrame([])
sub['id'] = test_id
sub['target'] = pred
sub.to_csv('submission.csv', index=False)